### TODO Recording:

- Set up the input/ folder, should be empty
- Delete the checkpoint folder

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder \
    .appName("StreamingTableDemo") \
    .getOrCreate()

25/02/13 08:44:23 WARN Utils: Your hostname, Jananis-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.68.52 instead (on interface en0)
25/02/13 08:44:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/13 08:44:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql.types import StructType, StructField, IntegerType, \
    StringType, DoubleType

schema = StructType([
    StructField("Rank", IntegerType(), True),
    StructField("Name", StringType(), True),
    StructField("Manufacturer", StringType(), True),
    StructField("Country", StringType(), True),
    StructField("Year", IntegerType(), True),
    StructField("Segment", StringType(), True),
    StructField("Total_Cores", IntegerType(), True),
    StructField("Processor_Speed", IntegerType(), True),
    StructField("CoProcessor_Cores", IntegerType(), True),
    StructField("Rmax", DoubleType(), True),
    StructField("Rpeak", DoubleType(), True),
    StructField("Power", DoubleType(), True),
    StructField("Power_Efficiency", DoubleType(), True),
    StructField("Architecture", StringType(), True),
    StructField("Processor_Technology", StringType(), True),
    StructField("Operating_System", StringType(), True),
    StructField("OS_Family", StringType(), True),
])

In [3]:
streaming_df = spark.readStream \
    .format("csv") \
    .option("header", "true") \
    .schema(schema) \
    .load("input/")

selected_df = streaming_df.select(
    "Name", "Manufacturer", "Country", "Year", "Segment", "Power", "Architecture"
).filter(col("Segment") == "Vendor")

In [4]:
selected_df.writeStream \
    .option("checkpointLocation", "checkpoint_orig/") \
    .toTable("supercomputers_streaming_table")

25/02/13 08:44:27 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


### TODO Recording:

- Show the spark_warehouse/ folder created in the directory
- Show the contents of this folder

In [6]:
spark.read.table("supercomputers_streaming_table").show(n=100)

+--------------------+---------------+-------------+----+-------+------+------------+
|                Name|   Manufacturer|      Country|Year|Segment| Power|Architecture|
+--------------------+---------------+-------------+----+-------+------+------------+
|              Selene|         Nvidia|United States|2020| Vendor|2646.0|     Cluster|
|        Voyager-EUS2|Microsoft Azure|United States|2021| Vendor|  NULL|     Cluster|
|         Pioneer-EUS|Microsoft Azure|United States|2020| Vendor|  NULL|     Cluster|
|        Pioneer-SCUS|Microsoft Azure|United States|2021| Vendor|  NULL|     Cluster|
|         Pioneer-WEU|Microsoft Azure|  Netherlands|2021| Vendor|  NULL|     Cluster|
|NVIDIA Cambridge-...|         Nvidia|United States|2021| Vendor|  NULL|     Cluster|
+--------------------+---------------+-------------+----+-------+------+------------+



### TODO Recording:

- Add a file to the input/ folder
- Re-run the previous cell - should have contents in the streaming table

In [7]:
spark.sql("DESCRIBE EXTENDED supercomputers_streaming_table").show(truncate=False)

+----------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                                                                                                            |comment|
+----------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+
|Name                        |string                                                                                                                                                               |NULL   |
|Manufacturer                |string                                                                                                                                                

Read from the streaming table

In [8]:
filtered_df = spark.readStream \
    .table("supercomputers_streaming_table")\
    .filter(col("Year") >= 2020)

In [9]:
filtered_df\
    .writeStream.option("checkpointLocation", "checkpoint_new/")\
    .format("console") \
    .outputMode("append") \
    .option("truncate", False) \
    .start()

25/02/13 08:44:49 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-------------------------------+---------------+-------------+----+-------+------+------------+
|Name                           |Manufacturer   |Country      |Year|Segment|Power |Architecture|
+-------------------------------+---------------+-------------+----+-------+------+------------+
|Selene                         |Nvidia         |United States|2020|Vendor |2646.0|Cluster     |
|Voyager-EUS2                   |Microsoft Azure|United States|2021|Vendor |NULL  |Cluster     |
|Pioneer-EUS                    |Microsoft Azure|United States|2020|Vendor |NULL  |Cluster     |
|Pioneer-SCUS                   |Microsoft Azure|United States|2021|Vendor |NULL  |Cluster     |
|Pioneer-WEU                    |Microsoft Azure|Netherlands  |2021|Vendor |NULL  |Cluster     |
|NVIDIA Cambridge-1 DGX SuperPOD|Nvidia         |United States|2021|Vendor |NULL  |Cluster     |
+-----------------------------

### TODO Recording:

- Add files 2,3,4,5 to the input/ folder and watch the console output here